In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
train_data=pd.read_excel('meta_list_labeling.xlsx')
train_data=train_data[train_data['has_lyric']==1]
train_data=train_data[:50]

In [4]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [5]:
max_seq_len = 128

In [6]:
def convert_tracks_to_features(tracks,labels,max_seq_len,tokenizer):
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [] , []
    idx_encode = preprocessing.LabelEncoder()
    idx_encode.fit(labels)
    data_labels=idx_encode.transform(labels)
    
    label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
    idx_label = {value: key for key, value in label_idx.items()}
    
    for track, label in tqdm(zip(tracks,labels),total=len(tracks)):
        
        input_id=tokenizer.encode(track,max_length=max_seq_len,pad_to_max_length=True)
        padding_count = input_id.count(tokenizer.pad_token_id)
        attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
        
        # token_type_id는 세그먼트 임베딩을 위한 것으로 이번 예제는 문장이 1개이므로 전부 0으로 통일.
        token_type_id = [0] * max_seq_len
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)

    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels,label_idx,idx_label

In [8]:
train_X, train_y,label_idx,idx_label  = convert_tracks_to_features(train_data['lyric'], train_data['mood'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\Users\aa823\miniconda3\envs\nlp\lib\site-packages\transformers\tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g.

In [9]:
model = TFBertModel.from_pretrained("bert-base-multilingual-cased")

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [10]:
class TFBertForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForSequenceClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.models.Sequential()
        self.classifier.add(tf.keras.layers.Dense(16, activation='relu'))
        self.classifier.add(tf.keras.layers.Dense(32, activation='relu'))
        self.classifier.add(tf.keras.layers.Dense(16, activation='relu'))
        self.classifier.add(tf.keras.layers.Dense(3,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier'))
        

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [11]:
model = TFBertForSequenceClassification("bert-base-multilingual-cased")
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already

In [16]:
model.fit(train_X, train_y, epochs=10, batch_size=8, validation_split=0.2)

Epoch 1/10
5/5 [==============================] - 0s 84ms/step - loss: 0.8509 - accuracy: 0.7000 - val_loss: 1.1577 - val_accuracy: 0.5000
Epoch 2/10
5/5 [==============================] - 0s 80ms/step - loss: 0.8225 - accuracy: 0.6000 - val_loss: 1.1234 - val_accuracy: 0.6000
Epoch 3/10
5/5 [==============================] - 0s 82ms/step - loss: 0.7744 - accuracy: 0.7250 - val_loss: 1.1479 - val_accuracy: 0.6000
Epoch 4/10
5/5 [==============================] - 0s 80ms/step - loss: 0.7316 - accuracy: 0.8000 - val_loss: 1.1566 - val_accuracy: 0.5000
Epoch 5/10
5/5 [==============================] - 0s 80ms/step - loss: 0.7374 - accuracy: 0.7750 - val_loss: 1.1403 - val_accuracy: 0.5000
Epoch 6/10
5/5 [==============================] - 0s 80ms/step - loss: 0.7476 - accuracy: 0.7250 - val_loss: 1.1328 - val_accuracy: 0.6000
Epoch 7/10
5/5 [==============================] - 0s 82ms/step - loss: 0.7806 - accuracy: 0.8000 - val_loss: 1.1822 - val_accuracy: 0.2000
Epoch 8/10
5/5 [===========

In [17]:
def sentiment_predict(new_sentence):
    input_id = tokenizer.encode(new_sentence, max_length=max_seq_len, pad_to_max_length=True)

    padding_count = input_id.count(tokenizer.pad_token_id)
    attention_mask = [1] * (max_seq_len - padding_count) + [0] * padding_count
    token_type_id = [0] * max_seq_len

    input_ids = np.array([input_id])
    attention_masks = np.array([attention_mask])
    token_type_ids = np.array([token_type_id])

    encoded_input = [input_ids, attention_masks, token_type_ids]
    score = model.predict(encoded_input)
    idx=np.argmax(score)
    return idx,score

In [18]:
test=pd.read_excel('meta_list_labeling.xlsx')
test=test[test['has_lyric']==1]
for i in range(101,300):
    
    lyric=test['lyric'][i]
    idx,score=sentiment_predict(lyric)
    sentiment=idx_label[idx]
    print(test['track_title'][i])
    print(sentiment)
    print(score[0][idx])

1/1 [==============================] - 0s 23ms/step
삐삐
gloomy
0.7981874
1/1 [==============================] - 0s 24ms/step
소녀
relaxed
0.41803378
1/1 [==============================] - 0s 23ms/step
Dance The Night Away
delighted
0.5812669
1/1 [==============================] - 0s 24ms/step
IDOL
gloomy
0.79787886
1/1 [==============================] - 0s 23ms/step
한숨
delighted
0.5785353
1/1 [==============================] - 0s 22ms/step
그대네요 (With 아이유)
delighted
0.58177024
1/1 [==============================] - 0s 22ms/step
Let It Go (From "Frozen"/Soundtrack Version)
gloomy
0.6857783
1/1 [==============================] - 0s 23ms/step
썸 (Feat. 릴보이 of 긱스)
gloomy
0.79903126
1/1 [==============================] - 0s 23ms/step
만약에
delighted
0.57948786
1/1 [==============================] - 0s 22ms/step
Something Just Like This
delighted
0.5769166
1/1 [==============================] - 0s 23ms/step
Santa Tell Me
delighted
0.5541659
1/1 [==============================] - 0s 23ms/step
Rollin

1/1 [==============================] - 0s 23ms/step
양화대교
gloomy
0.79846424
1/1 [==============================] - 0s 23ms/step
시차 (We Are) (Feat. 로꼬 & GRAY)
gloomy
0.78602225
1/1 [==============================] - 0s 23ms/step
나만 몰랐던 이야기
delighted
0.5741973
1/1 [==============================] - 0s 23ms/step
넘쳐흘러
gloomy
0.78831166
1/1 [==============================] - 0s 22ms/step
Call You Mine (Feat. Geologic of The Blue Scholars)
gloomy
0.7857047
1/1 [==============================] - 0s 24ms/step
Roller Coaster
delighted
0.575903
1/1 [==============================] - 0s 23ms/step
오래된 노래
relaxed
0.4182508
1/1 [==============================] - 0s 23ms/step
All For You
gloomy
0.7986778
1/1 [==============================] - 0s 23ms/step
이쁘다니까
gloomy
0.7976066
1/1 [==============================] - 0s 22ms/step
Maps
gloomy
0.7928686
1/1 [==============================] - 0s 22ms/step
제자리 걸음
gloomy
0.79700315
1/1 [==============================] - 0s 23ms/step
눈사람
relaxed
0.4177796
1/

1/1 [==============================] - 0s 22ms/step
우주를 건너
delighted
0.5812267
1/1 [==============================] - 0s 24ms/step
Chandelier
gloomy
0.7880563
